In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from tqdm import tqdm
import warnings
from datetime import datetime, timedelta
import statsmodels.api as sm
import os
warnings.filterwarnings("ignore")
os.getcwd()

'/Users/test/risksoft/TS/db_py'

In [2]:
try:
    os.chdir("../test123")
except FileNotFoundError:
    os.makedirs("../test123")
    os.chdir("../test123")

In [182]:
directory = 'tw/price'
list_ = os.listdir(directory)
df_list = [pd.read_parquet(os.path.join(directory, i)) for i in list_]
df_all = pd.concat(df_list)
df_all['da1'] = df_all['da'].apply(lambda x: f"{int(x[:3])+1911}-{x[4:6]}-{x[7:9]}")
df = df_all.drop_duplicates(subset=['ticker', 'da1'])
df['da_check'] = df['da1'].astype(str).apply(lambda x: int(x[:4]))
df_final = df[df['da_check'].isin([i for i in range(2018, 2026)])]

for values in ['cl', 'op', 'vol(volume)', 'vol(turnover)']:
    df_pivot = df_final.pivot(values=values, columns='ticker', index='da1')
    df_pivot.replace("--", None, inplace=True)
    df_pivot.replace("-", None, inplace=True)
    df_pivot.ffill(inplace=True)
    df_pivot = df_pivot.loc['2018-01-01':(datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d")]
    df_pivot.sort_index(ascending=True, inplace=True)
    df_pivot = df_pivot.apply(lambda x: x.str.replace(',', '').astype(float))
    df_pivot.dropna(how='all', axis=1, inplace=True)
    df_pivot.to_parquet(f"tw/pdata/{values}.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}.parquet successful")
    df_pivot_pct = df_pivot.pct_change().dropna()
    df_pivot.to_parquet(f"tw/pdata/{values}_pct.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}_pct.parquet successful")
    


store pdata of values cl to tw/pdata/cl.parquet successful
store pdata of values cl to tw/pdata/cl_pct.parquet successful
store pdata of values op to tw/pdata/op.parquet successful
store pdata of values op to tw/pdata/op_pct.parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume).parquet successful
store pdata of values vol(volume) to tw/pdata/vol(volume)_pct.parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover).parquet successful
store pdata of values vol(turnover) to tw/pdata/vol(turnover)_pct.parquet successful


In [127]:
directory = 'tw/pb_ratio'
list_ = os.listdir(directory)
df_list = [pd.read_parquet(os.path.join(directory, i)) for i in list_]
df_all = pd.concat(df_list)
df_all['da'] = df_all.index
df = df_all.drop_duplicates(subset=['ticker', 'da'])
for values in ['pe', 'pb']:
    df_pivot = df.pivot(values=values, columns='ticker', index='da')

    df_pivot.replace("--", None, inplace=True)
    df_pivot.replace("-", None, inplace=True)
    df_pivot.ffill(inplace=True)
    df_pivot = df_pivot.loc['2018-01-01':(datetime.now()+timedelta(days=1)).strftime("%Y-%m-%d")]
    df_pivot.sort_index(ascending=True, inplace=True)
    df_pivot = df_pivot.apply(lambda x: x.str.replace(',', '').astype(float))
    df_pivot.dropna(how='all', axis=1, inplace=True)
    df_pivot.to_parquet(f"tw/pdata/{values}.parquet")
    print(f"store pdata of values {values} to tw/pdata/{values}.parquet successful")


store pdata of values pe to tw/pdata/pe.parquet successful
store pdata of values pb to tw/pdata/pb.parquet successful


### 富櫃50 TPEX

In [ ]:

import requests
list_list = []
for year in range(113, 102, -1):
    limit_month = 8 if year == 113 else 13
    for index, month in enumerate(range(1, limit_month)):
        month = f"0{month}" if month < 10 else month
        url_gretai50 = f"https://www.tpex.org.tw/web/stock/iNdex_info/gretai50/inxhis/rihisqry_result.php?l=zh-tw&d={year}/{month}/01&_=1720688879318"
        response = requests.get(url_gretai50)
        data = response.json()['aaData']
        list_list.append(data)
list_ = sum(list_list, [])

In [32]:
df = pd.DataFrame(list_)
df.to_parquet('../db/tw/ind/gretai50.parquet')

### 櫃買指數